In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install langgraph langchain-community langchain-core networkx langchain_experimental langchain-openai chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5

In [3]:
import os
import getpass
import networkx as nx
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import GraphQAChain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
# Set your OpenAI API key (replace with your actual key)
if not os.environ.get('OPENAI_API_KEY'):
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OPENAI Key')

In [6]:
# Create Knowledge Graph in Memory 

def create_knowledge_graph():
    """Creates a simple NetworkX knowledge graph in memory."""
    G = nx.Graph()
    # Add nodes and edges representing knowledge graph
    G.add_edge("John", "Software Engineer", relation="works as")
    G.add_edge("John", "Microsoft", relation="works at")
    G.add_edge("Microsoft", "Multi National Company", relation="is a")
    G.add_edge("Bob", "John", relation="reports to")
    # G.add_edge("Bob", "Acme Corp", relation="")
    return G

# Convert Graph to Text

def graph_to_text(graph):
    """Converts a NetworkX graph to a string representation suitable for RAG."""
    text = ""
    for u, v, data in graph.edges(data=True):
        text += f"{u}, {data['relation']}, {v}\n"
    return text


# Index the Knowledge Graph (using Vectorstore)

def create_vectorstore_from_text(text):
    """Creates a Chroma vectorstore from the knowledge graph text."""
    text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    docs = text_splitter.split_text(text)  # Split the text directly

    embeddings = OpenAIEmbeddings()
    db = Chroma.from_texts(docs, embeddings)  # from_texts takes a list of strings
    return db


# Create RAG Chain

def create_rag_chain(vectorstore):
    """Creates a RetrievalQA chain using the provided vectorstore."""

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    prompt_template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    {context}

    Question: {question}
    Helpful Answer:"""
    QA_PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": QA_PROMPT},
        return_source_documents=True
    )

    return qa_chain


# Main Function

def main():
    # Create Knowledge Graph
    graph = create_knowledge_graph()

    # Convert Graph to Text
    kg_text = graph_to_text(graph)

    # Index the Knowledge Graph (using Vectorstore)
    db = create_vectorstore_from_text(kg_text)

    # Create the RAG Chain
    rag_chain = create_rag_chain(db)

    # Query the RAG Chain
    query = "Where does Bob work?"
    print(f"Query: {query}")
    result = rag_chain.invoke(query)
    print(f"Answer: {result['result']}")
    print(f"Source Documents: {result.get('source_documents')}")

    query = "What type of company is Bob working?"
    print(f"Query: {query}")
    result = rag_chain(query)
    print(f"Answer: {result['result']}")
    print(f"Source Documents: {result.get('source_documents')}")



if __name__ == "__main__":
    main()

Query: Where does Bob work?
Answer: Bob works at Microsoft.
Source Documents: [Document(metadata={}, page_content='John, works as, Software Engineer\nJohn, works at, Microsoft\nJohn, reports to, Bob\nMicrosoft, is a, Multi National Company'), Document(metadata={}, page_content='John, works as, Software Engineer\nJohn, works at, Microsoft\nJohn, reports to, Bob\nMicrosoft, is a, Multi National Company'), Document(metadata={}, page_content='John, works as, Software Engineer\nJohn, works at, Microsoft\nJohn, reports to, Bob\nMicrosoft, is a, Multi National Company')]
Query: What type of company is Bob working?
Answer: Since Bob is John's superior at Microsoft, Bob is also working at a Multi National Company.
Source Documents: [Document(metadata={}, page_content='John, works as, Software Engineer\nJohn, works at, Microsoft\nJohn, reports to, Bob\nMicrosoft, is a, Multi National Company'), Document(metadata={}, page_content='John, works as, Software Engineer\nJohn, works at, Microsoft\nJohn